# Ficha de Análise Léxica (Lex)

## Introdução

A análise léxica é o processo de conversão de uma sequência de caracteres numa sequência de *tokens*, em que cada *token* representa uma unidade significativa da linguagem à qual os caracteres pertencem.

Em Python, podemos fazer análise léxica de várias formas. A que iremos utilizar nas aulas recorre ao módulo Ply, que para além de análise léxica vai-nos permitir fazer análise sintática.

Antes de usar o módulo Ply, precisamos de o instalar. Para isso, podemos usar o comando seguinte:

```sh
$ pip install ply
```

Depois, apenas precisamos de importar a ferramenta `lex.py` no nosso programa:

In [1]:
import ply.lex as lex

A primeira coisa que o nosso analisador léxico (ou *lexer*/*tokenizer*) precisa de ter é uma lista de *tokens*. Como exemplo, vamos definir um *lexer* que lê expressões aritméticas, como "4 * (2 + 3)". Neste exemplo já somos capazes de identificar alguns *tokens*...

In [ ]:
tokens = (
    'NUMBER',
    'PLUS',
    # completar...
)

A seguir é preciso especificar cada *token*. Por outras palavras, precisamos de definir expressões regulares que permitam ao *tokenizer* identificar os *tokens*. Podemos fazê-lo através de variáveis ou de funções.

In [ ]:
t_PLUS = r'\+'
# completar...

def t_NUMBER(t):
    # completar
    pass

Podemos especificar um conjunto de caracteres que o analisador léxico vai ignorar.

In [ ]:
t_ignore = ' \t\n'

Precisamos ainda de definir o comportamento do *tokenizer* caso encontre um carácter ou sequência de caracteres que não corresponda a nenhum *token* conhecido.

In [ ]:
def t_error(t):
    print(f"Carácter ilegal {t.value[0]}")
    t.lexer.skip(1)

Agora, já somos capazes de construir o nosso analisador léxico.

In [ ]:
lexer = lex.lex()

Para o usar, precisamos de lhe dar algum valor de *input* e depois pedir-lhe para ir devolvendo os *tokens* que encontrar.

In [ ]:
data = '''
3 + 4 * 10
  + -20 *2
'''

lexer.input(data)

while True:
    tok = lexer.token()
    if not tok:
        break
    print(tok)

É possível consultar a documentação do *lex.py* em https://ply.readthedocs.io/en/latest/ply.html#lex. 

## Exercícios

### 1. Frases

Define um analisador léxico capaz de ler uma frase e de identificar os seus componentes (palavras, vírgulas, sinais de pontuação).

In [1]:
# ON FILE: ex1_fichaLex.py

### 2. Listas Mistas

Define um analisador léxico capaz de receber listas números, palavras ou valores booleanos como input (e.g.: `[ 1,5, palavra, False ,3.14,   0, fim]`) e identificar os seus *tokens*.

In [ ]:
# ON FILE: ex2_fichaLex.py

### 3. JSON

Define um analisador léxico capaz de ler ficheiros em formato JSON e identificar os seus *tokens*.

Exemplo de um documento JSON:

---

```json
{
  "name": "John Doe",
  "age": 21,
  "gender": "male",
  "height": 1.68,
  "address": {
    "street": "123 Main Street",
    "city": "New York",
    "country": "USA",
    "zip": "10001"
  },
  "married": false,
  "hobbies": [
    {
      "name": "reading",
      "books": [
        {
          "title": "Heartstopper: Volume 1",
          "author": "Alice Oseman",
          "genres": ["Graphic Novels", "Romance", "Queer"]
        },
        {
          "title": "1984",
          "author": "George Orwell",
          "genres": ["Science Fiction", "Dystopia", "Politics"]
        }
      ]
    },
    {
      "name": "gaming",
      "games": [
        {
          "title": "Portal 2",
          "platform": ["PC", "PlayStation 3", "Xbox 360"]
        },
        {
          "title": "Synth Riders", 
          "platform": ["PSVR", "PSVR2", "PCVR", "Oculus Quest"]
        }
      ]
    }
  ]
}
```
---

## Condições de contexto

Para certos analisadores léxico, pode ser útil ter diferentes estados. Por exemplo, se definirmos um analisador léxico para um ficheiro XML, pode ser útil verificar se o nome usado para fechar uma *tag* foi o mesmo que foi usado para a abrir.

Exemplo de parte de um ficheiro XML:

```xml
<pessoa>
    <nome>Maria</nome>
    <idade>32</idade>
</pessoa>
```

In [ ]:
import ply.lex as lex

states = (
    ('taga', 'exclusive'),
    ('tagf', 'exclusive'), # num estado exclusivo, apenas aplicamos os tokens e regras para esse estado
                           # por outro lado, num estado inclusivo, as regras e tokens desse estado juntam-se às outras regras e tokens
                           # o estado inicial chama-se 'INITIAL' e não é preciso defini-lo
)

tokens = (
    'ABRIR_TAG',
    'ABRIR_TAG_F',
    'FECHAR_TAG',
    'NOME_TAG',
    'VALOR'
)

t_ignore = ' \t\n' # estes tokens apenas são ignorados no estado 'INITIAL' e em estados inclusivos

t_VALOR = r'[^<]+'

def t_ABRIR_TAG_F(t):
    r'</'
    t.lexer.begin('tagf') # entramos no estado 'tagf'
    return t

def t_ABRIR_TAG(t):
    r'<'
    t.lexer.begin('taga') # entramos no estado 'taga'
    return t

def t_taga_tagf_FECHAR_TAG(t):
    r'>'
    t.lexer.begin('INITIAL') # voltamos ao estado inicial
    return t

def t_taga_NOME_TAG(t):
    r'\w+'
    t.lexer.stack.append(t.value)
    return t

def t_tagf_NOME_TAG(t):
    r'\w+'
    if len(t.lexer.stack) > 0:
        if (nt := t.lexer.stack.pop(-1)) != t.value:
            print(f"Erro - esperado nome de tag '{nt}', mas foi lido '{t.value}'!")
    else:
        print("Erro - nenhuma tag aberta!")
    return t

def t_ANY_error(t): # regra válida para todos os estados
    print(f"Carácter ilegal: {t.value[0]}")
    t.lexer.skip(1)


data = '''
<pessoa>
    <nome>Maria</nome>
    <idade>32</idade>
</pessoa>
'''

lexer = lex.lex()

lexer.stack = list() # vamos usar esta lista como stack para verificar os nomes das tags

lexer.input(data)

while True:
    tok = lexer.token()
    if not tok:
        break
    print(tok)

Se quisermos manter informação sobre as linhas nas quais os *tokens* aparecem, podemos usar o atributo `lineno`.

In [ ]:
t_ignore = r' \t'

def t_newline(t):
    r'\n+'
    t.lexer.lineno += len(t.value)

## Exercícios 2

### 1. BibTeX

Define um analisador léxico capaz de ler um ficheiro no formato *BibTeX* e identificar os seus *tokens*.

Exemplo de um ficheiro BibTeX:

---

```bibtex
@incollection {HDYE78,
author = "Ricardo Martini and Pedro Rangel Henriques and Giovani Libreloto",
title = "Storing Archival Emigration Documents to Create Virtual Exhibition Rooms",
booktitle = "New Contributions in Information Systems and Technologies",
series="Advances in Intelligent Systems and Computing",
editor="Rocha, Alvaro and Correia, Ana and Costanzo, S. and Reis, Luis Paulo",
volume="353",
pages="403-409",
year = "2015",
month =  "April"
}


@book {H787,
author = {Vitor T. Martins and Pedro Rangel Henriques and Daniela da Cruz},
title = {An AST-based tool, Spector, for Plagiarism Detection},
booktitle = {Proceedings of SLATE’15},
pages = {173--178},
ISBN = {},
year = {2015},
month =   {},
publisher = {Fundacion General UCM},
annote = {Keywords: software, plagiarism, detection, comparison, test}}

@book {H787,
author = {Vitor T. Martins and Pedro Rangel Henriques and Daniela da Cruz},
title = "{A}n {AST}-based tool, {S}pector, for Plagiarism Detection",
booktitle = {Proceedings of SLATE’15},
pages = {173--178},
ISBN = {},
year = {2015},
month =   {},
publisher = {Fundacion General UCM},
annote = {Keywords: software, plagiarism, detection, comparison, test}
}
```
---

### 2. Markdown

Define um analisador léxico capaz de ler ficheiros em formato Markdown e identificar os seus *tokens*. O *tokenizer* deve conseguir identificar pelo menos:

- cabeçalhos
- parágrafos
- listas
- texto itálico
- texto negrito
- blocos de código
- citações

Exemplo de um documento Markdown:

---

````markdown
# This is a heading

## This is a subheading

This is some **bold** text.

This is some *italic* text.

- This is a bullet point
- This is another bullet point

1. This is a numbered list
2. This is another numbered list item

> This is a blockquote.

`This is some inline code.`

```python
# This is some code block
print("Hello, world!")
```
````
---

In [ ]:
## NOT COMPLETE, on file exMD_fichaLex.py

### 3. Somador on/off

Cria um programa em Python que tenha o seguinte comportamento:

* Pretende-se um programa que some todas as sequências de dígitos que encontre num texto;
* Prepara o programa para ler o texto do canal de entrada: stdin;
* Sempre que encontrar a string “Off” em qualquer combinação de maiúsculas e minúsculas, esse comportamento é desligado;
* Sempre que encontrar a string “On” em qualquer combinação de maiúsculas e minúsculas, esse comportamento é novamente ligado;
* Sempre que encontrar o caráter “=”, o resultado da soma é colocado na saída.

Este exercício já foi proposto como TPC, mas agora deves tentar resolvê-lo usando um analisador léxico com condições de contexto.